In [1]:
using MagneticLaplacianSparsifier
using Graphs, MetaGraphs, Random, IterTools, LinearAlgebra, GraphPlot, Arpack, Plots, SparseArrays
using Statistics
using Measures
using StatsBase

In [11]:
# graph generation
q = 0;
n = 300;
eta = 0.5;
p = 0.9;

rng = Random.default_rng();

# planted ranking score
planted_score = randperm(rng,n)

n_rep = 1

off_diag_chol_custom = zeros(n_rep,1)
off_diag_chol_random = zeros(n_rep,1)
off_diag_chol_auto = zeros(n_rep,1)
chol_bound = zeros(n_rep,1)

for i= 1:n_rep
    comp_graph = gen_graph_ero(rng, n, p, eta;planted_score)

    # sample a CRSF
    crsf = multi_type_spanning_forest(rng, comp_graph, q)

    # get the nodes in the cycle(s)
    cycles = get_prop(crsf,:cycle_nodes)

    # # get the branches in the (reverse) order there were sampled
    branches = get_prop(crsf,:branches)

    flt_branches = collect(Iterators.flatten(branches))
    flt_cycles = collect(Iterators.flatten(cycles))

    # indices array putting the nodes in the right order
    ind = [flt_branches; setdiff(1:n,flt_branches)];
    # construct Laplacian
    B = sp_magnetic_incidence(crsf; oriented=true);
    L = B*B';

    # find cholesky
    lower_factor = cholesky(L, perm = ind).L;

    chol_matrix = sparse(lower_factor)
    # number of diag entries

    off_diag_chol_custom[i] = nnz(chol_matrix) - n

    print("choleski of L with custom perm: ", off_diag_chol_custom[i], " off diagonal entries \n")

    # upper bound on number of non-zero entries (Prop 2 in paper)
    # n + sum_{cycle c}(n_c - 3)

    bound = isempty(cycles) ? 0 : sum(length(c) - 3 for c in cycles)
    bound += nv(crsf)

    print("theoretical upper bound: ", bound, " off diagonal entries \n")

    chol_bound[i] = bound

    # cholesky of random permuted Laplacian
    perm = randperm(n);
    lower_factor_rand_perm = sparse(cholesky(L, perm = perm).L);

    off_diag_chol_random[i] = nnz(lower_factor_rand_perm) - n

    print("choleski of L with RANDOM permutation: ", off_diag_chol_random[i], " off diagonal entries \n")

    # cholesky of raw Laplacian
    lower_factor_raw = sparse(cholesky(L).L);

    off_diag_chol_auto[i] = nnz(lower_factor_raw) - n

    print("choleski of L with AUTOMATIC permutation: ", off_diag_chol_auto[i], " off diagonal entries \n")

end



choleski of L with custom perm: 314.0 off diagonal entries 
theoretical upper bound: 314 off diagonal entries 
choleski of L with RANDOM permutation: 1231.0 off diagonal entries 
choleski of L with AUTOMATIC permutation: 314.0 off diagonal entries 
